Following this course: https://github.com/louthy/language-ext/wiki/Thinking-Functionally    

In [ ]:
// quick setup
#r "nuget:Shouldly"
#r "nuget:LanguageExt.Core"
using Shouldly;
using LanguageExt;

Installed Packages LanguageExt.Core, 4.2.9 Shouldly, 4.0.3

### Sarting simple
`AddOne :: int -> int`    
A function "maps" a value in the "domain" to a value in the "range". There is no "computation".

In [ ]:
int AddOne(int x) => x + 1;
return AddOne(5);

6

#### Important notes:    
`x` is not something that can change once we pass 5 in.    
This is not assignment - What is happening here is "binding".    
Once a domain value is "bound" is cannot be changed.     
`x` is a placeholder that can be referred to later, not changed.    
**There are no "variables", only values**

_side note, yes x is a variable being assigned a value - but it is forbidden 'functionaly'_

### Function Values
`Func<domain, range>`    

Similarly - the name `AddOne` is just a "binding" to "the function that adds one to its input"    
"Every time you see the name `AddOne`, replace it with the function that adds one to its input".    

For example, we can "bind" `AddOne` to a new name.    
`PlusOne` and `AddOne` are "bound" to the same value, which is the function that adds one to its input.    

In [ ]:
Func<int, int> PlusOne = AddOne;
return (PlusOne(5), AddOne(5));

Item1,Item2
6,6


### Simple Value
Imagine an operation that always returns `5` - this is a constant.    
in "C#" you can easily define that with the following: `const int C = 5;` or `static int C => 5; //Func<int>`.      
simple values do not need to be evaluated once "bound" - This is the subtle difference between them and function values, otherwise they are both values of Func.

With this, `5` and `Five` are both just values that can be passed around. - This is a key part of thinking functionaly: functions are values that can be passed around to other functions.

A constant function for `5` would be defined as:    
`Func<Unit, int>`   

In [ ]:
const int C = 5;
//or
static int C_alt => 5;
//or
static int Five_alt(Unit _) => 5;
//or
Func<Unit, int> Five = _ => 5;

return (AddOne(C), AddOne(C_alt), AddOne(Five(Unit.Default)), AddOne(Five_alt(Unit.Default)));

Item1,Item2,Item3,Item4
6,6,6,6


### "Objects" vs "Values"

Values:
- A member of a domain. The domain of ints, strings, functions that map ints to strings, etc...    
- They are immutable
- They have no behaviour

Objects: 
- Encapsulation of data with associated behaviour
- Mutable

Avoid using "object" for standard values for functional C#.    
It should only be used to refer to true classes, or values that expose member functions.

### How types work with functions
